# T81-558: Applications of Deep Neural Networks
**Module 7: Generative Adversarial Networks**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 7 Material

* Part 7.1: Introduction to GANS for Image and Data Generation [[Video]](https://www.youtube.com/watch?v=u8xn393mDPM&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN&index=21) [[Notebook]](t81_558_class_07_1_gan_intro.ipynb)
* Part 7.2: Implementing a GAN in Keras [[Video]](https://www.youtube.com/watch?v=cf6FDLFNWEk&index=22&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_07_2_Keras_gan.ipynb)
* **Part 7.3: Face Generation with StyleGAN and Python** [[Video]](https://www.youtube.com/watch?v=LSSH_NdXwhU&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN&index=23) [[Notebook]](t81_558_class_07_3_style_gan.ipynb)
* Part 7.4: GANS for Semi-Supervised Learning in Keras [[Video]](https://www.youtube.com/watch?v=LSSH_NdXwhU&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN&index=23) [[Notebook]](t81_558_class_07_4_gan_semi_supervised.ipynb)
* Part 7.5: An Overview of GAN Research [[Video]](https://www.youtube.com/watch?v=LSSH_NdXwhU&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN&index=23)  [[Notebook]](t81_558_class_07_5_gan_research.ipynb)


# Part 7.3: Face Generation with StyleGAN and Python

GANs have appeared frequently in the media, showcasing their ability to generate extremely photorealistic faces.  One significant step forward for realistic face generation was nVidia StyleGAN, which was introduced in the following paper.

* Karras, T., Laine, S., & Aila, T. (2018). [A style-based generator architecture for generative adversarial networks](https://arxiv.org/abs/1812.04948). *arXiv preprint arXiv:1812.04948*.

In this part we will make use of StyleGAN.  We will also preload weights that nVidia trained on.  This will allow us to generate high resolution photorealistic looking faces, such as this one.

![GAN](https://raw.githubusercontent.com/jeffheaton/t81_558_deep_learning/master/images/gan-example246.png "GAN")

The above image was generated with StyleGAN, using Google CoLab.  Following the instructions in this section, you will be able to create faces like this of your own.  

While the above image looks much more realistic than the previous set of images, it is not perfect.  There are usually a number of tell-tail signs that you are looking at a computer generated image.  One of the most obvious is usually the surreal, dream-like backgrounds.  The background does not look obviously fake, at first glance; however, upon closer inspection you usually can't quite discern exactly what a GAN generated background actually is.  Also look at the image character's left eye.  It is slightly unrealistic looking, especially near the eyelashes.

Look at the following GAN face.  Can you spot any imperfections?

![GAN](https://raw.githubusercontent.com/jeffheaton/t81_558_deep_learning/master/images/gan-example221.png "GAN")

Notice the earrings?  GANs sometimes have problems with symmetry, particularly earrings.

### Keras Sequence vs Functional Model API

Most of the neural networks create in this course have made use of the Keras sequence object.  You might have noticed that we briefly made use of another type of neural network object for the ResNet, the Model.  These are the [two major means](https://keras.io/getting-started/functional-api-guide/) of constructing a neural network in Keras:

* [Sequential](https://keras.io/getting-started/sequential-model-guide/) - Simplified interface to Keras that supports most models where the flow of information is a simple sequence from input to output. 
* [Keras Functional API](https://keras.io/getting-started/functional-api-guide/) - More complex interface that allows neural networks to be constructed of reused layers, multiple input layers, and supports building your own recurrent connections.

It is important to point out that these are not two specific types of neural network.  Rather, they are two means of constructing neural networks in Keras.  Some types of neural network can be implemented in either, such as dense feedforward neural networks (like we used for the Iris and MPG datasets).  However, other types of neural network, like ResNet and GANs can only be used in the Functional Model API.

### Generating High Rez GAN Faces with Google CoLab

This notebook demonstrates how to run [NVidia StyleGAN](https://github.com/NVlabs/stylegan) inside of a Google CoLab notebook.  I suggest you use this to generate GAN faces from a pretrained model.  If you try to train your own, you will run into compute limitations of Google CoLab.

Make sure to run this code on a GPU instance.  GPU is assumed.

First, map your G-Drive, this is where your GANs will be written to.

In [ ]:
# Run this for Google CoLab
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Next, clone StyleGAN from GitHub.

In [ ]:
!git clone https://github.com/NVlabs/stylegan.git

Verify that StyleGAN has been cloned.

In [ ]:
!ls /content/stylegan/

Add the StyleGAN folder to Python so that you can import it.

In [ ]:
import sys
sys.path.insert(0, "/content/stylegan")

import dnnlib

The code below is based on code from NVidia. This actually generates your images.

In [ ]:
# Copyright (c) 2019, NVIDIA CORPORATION. All rights reserved.
#
# This work is licensed under the Creative Commons Attribution-NonCommercial
# 4.0 International License. To view a copy of this license, visit
# http://creativecommons.org/licenses/by-nc/4.0/ or send a letter to
# Creative Commons, PO Box 1866, Mountain View, CA 94042, USA.

"""Minimal script for generating an image using pre-trained StyleGAN generator."""

import os
import pickle
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import config

def main():
    # Initialize TensorFlow.
    tflib.init_tf()

    # Load pre-trained network.
    url = 'https://drive.google.com/uc?id=1MEGjdvVpUsu1jB4zrXZN7Y4kBBOzizDQ' # karras2019stylegan-ffhq-1024x1024.pkl
    with dnnlib.util.open_url(url, cache_dir=config.cache_dir) as f:
        _G, _D, Gs = pickle.load(f)
        # _G = Instantaneous snapshot of the generator. Mainly useful for resuming a previous training run.
        # _D = Instantaneous snapshot of the discriminator. Mainly useful for resuming a previous training run.
        # Gs = Long-term average of the generator. Yields higher-quality results than the instantaneous snapshot.

    # Print network details.
    Gs.print_layers()

    # Pick latent vector.
    rnd = np.random.RandomState()
    

    latents = rnd.randn(1, Gs.input_shape[1])

    # Generate image.
    fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    images = Gs.run(latents, None, truncation_psi=0.7, randomize_noise=True, output_transform=fmt)

    # Save image.
    os.makedirs(config.result_dir, exist_ok=True)
    png_filename = os.path.join(config.result_dir, f'/content/drive/My Drive/images/example1.png')
    PIL.Image.fromarray(images[0], 'RGB').save(png_filename)

if __name__ == "__main__":
    main()

# Part 7.4: GANS for Semi-Supervised Training in Keras

GANs can also be used to implement semi-supervised learning/training.  Normally GANs implement un-supervised training.  This is because there are no y's (expected outcomes) provided in the dataset.  The y-values are usually called labels.  For the face generating GANs, there is typically no y-value, only images.  This is unsupervised training.  Supervised training occurs when we are training a model to 

![GAN](https://raw.githubusercontent.com/jeffheaton/t81_558_deep_learning/master/images/gan-training.png "GAN")

The following paper describes the application of GANs to semi-supervised training.

* [Odena, A. (2016). Semi-supervised learning with generative adversarial networks. *arXiv preprint* arXiv:1606.01583.](https://arxiv.org/abs/1606.01583)

As you can see, supervised learning is where all data have labels.  Supervised learning attempts to learn the labels from the training data to predict these labels for new data.  Un-supervised learning has no labels and usually simply clusters the data or in the case of a GAN, learns to produce new data that resembles the training data.  Semi-supervised training has a small number of labels for mostly unlabeled data.  Semi-supervised learning is usually similar to supervised learning in that the goal is ultimately to predict labels for new data.

Traditionally, unlabeled data would simply be discarded if the overall goal was to create a supervised model.  However, the unlabeled data is not without value.  Semi-supervised training attempts to use this unlabeled data to help learn additional insights about what labels we do have.  There are limits, however.  Even semi-supervised training cannot learn entirely new labels that were not in the training set.  This would include new classes for classification or learning to predict values outside of the range of the y-values.

Semi-supervised GANs can perform either classification or regression.  Previously, we made use of the generator and discarded the discriminator.  We simply wanted to create new photo-realistic faces, so we just needed the generator.  Semi-supervised learning flips this, as we now discard the generator and make use of the discriminator as our final model.

### Semi-Supervised Classification Training

The following diagram shows how to apply GANs for semi-supervised classification training.

![GAN](https://raw.githubusercontent.com/jeffheaton/t81_558_deep_learning/master/images/gan-semi-class.png "GAN")

Semi-supervised classification training is laid exactly the same as a regular GAN.  The only differences is that it is not a simple true/false classifier as was the case for image GANs that simply classified if the generated image was a real or fake.  The additional classes are also added.  Later in this module I will provide a link to an example of [The Street View House Numbers (SVHN) Dataset](http://ufldl.stanford.edu/housenumbers/).  This dataset contains house numbers, as seen in the following image.

![GAN](https://raw.githubusercontent.com/jeffheaton/t81_558_deep_learning/master/images/svhn_examples_new.png "GAN")

Perhaps all of the digits are not labeled.  The GAN is setup to classify a real or fake digit, just as we did with the faces.  However, we also expand upon the real digits to include classes 0-9.  The GAN discriminator is classifying between the 0-9 digits and also fake digits.  A semi-supervised GAN classifier always classifies to the number of classes plus one. The additional class indicates a fake classification. 

### Semi-Supervised Regression Training

The following diagram shows how to apply GANs for semi-supervised regression training.

![GAN](https://raw.githubusercontent.com/jeffheaton/t81_558_deep_learning/master/images/gan-semi-reg.png "GAN")

Neural networks can perform both classification and regression simultaneously, it is simply a matter of how the output neurons are mapped.  A hybrid classification-regression neural network simply maps groups of output neurons to be each of the groups of classes to be predicted, along with individual neurons to perform any regression predictions needed.

A regression semi-supervised GAN is one such hybrid.  The discriminator has two output neurons.  The first output neuron performs the requested regression prediction.  The second predicts the probability that the input was fake.

### Application of Semi-Supervised Regression

An example of using Keras for Semi-Supervised classification is provided here.

* [Semi-supervised learning with Generative Adversarial Networks (GANs)](https://towardsdatascience.com/semi-supervised-learning-with-gans-9f3cb128c5e)
* [Unsupervised Representation Learning with Deep Convolutional Generative Adversarial Networks](https://arxiv.org/abs/1511.06434)
* [The Street View House Numbers (SVHN) Dataset](http://ufldl.stanford.edu/housenumbers/)

# Part 7.5: An Overview of GAN Research

* [Keras Implementations of Generative Adversarial Networks](https://github.com/eriklindernoren/Keras-GAN)
* [Curated List of Awesome GAN Applications and Demo](https://github.com/nashory/gans-awesome-applications)

### Select Projects

* [Few-Shot Adversarial Learning of Realistic Neural Talking Head Models](https://arxiv.org/abs/1905.08233v1), [YouTube of Talking Heads](https://www.youtube.com/watch?v=p1b5aiTrGzY)
* [Pose Guided Person Image Generation](https://papers.nips.cc/paper/6644-pose-guided-person-image-generation.pdf)
* [Deep Fake](https://www.youtube.com/watch?v=cQ54GDm1eL0)





# Module 7 Assignment

You can find the first assignment here: [assignment 7](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/assignments/assignment_yourname_class7.ipynb)